# API

In [22]:
from sympy import *
from sympy.physics.quantum import *

In [23]:
n1,n2,n3=symbols('n_1,n_2,n_3')

In [24]:
g={i:Operator('\gamma_{}'.format(i)) for i in range(1,5)}

In [25]:
gb={i:Operator(r'\bar{{\gamma}}_{}'.format(i)) for i in range(1,5)}

In [26]:
c={i:Operator(r'c_{}'.format(i)) for i in range(1,5)}

In [27]:
cd={i:Dagger(Operator(r'c_{}'.format(i))) for i in range(1,5)}

In [28]:
def ea(n1,n2,n3):
    return ((1+n1)/(1-n1))**(1/4)* ((n2-I*n3)/sqrt(n2**2+n3**2))**(1/2)
    

In [29]:
def K(n1,n2,n3,op1,op2):
    expa=N(ea(n1,n2,n3))
    expma=1/expa
    return (1-n1**2)**(1/4)*( (expa+ expma)/2)- ((1-n1**2)**(1/4)*(expa-expma)/2  *I* op1 * op2 )

In [30]:
selfjoint=[(Dagger(gi),gi) for gi in g.values()]+[(Dagger(gi),gi) for gi in gb.values()]

In [31]:
squareone=[(gi**2,1) for gi in g.values()]+[(gi**2,1) for gi in gb.values()]

In [32]:
normalorder=[gi[i] for i in range(1,len(g)+1) for gi in [g, gb]]

In [33]:
normalorder2=[(normalorder[j]*normalorder[i],-normalorder[i]*normalorder[j]) for i in range(len(normalorder)) for j in range(i+1,len(normalorder))]

In [34]:
g2c=[(g[i],(c[i]+Dagger(c[i]))) for i in range(1,len(g)+1)]+[(gb[i],(Dagger(c[i])-c[i])/(I)) for i in range(1,len(gb)+1)]

In [35]:
squarezero=[(ci**2,0) for ci in c.values()] +[(Dagger(ci)**2,0) for ci in c.values()]

In [36]:
normalorder_C=[Dagger(ci) for ci in c.values()]+[ci for ci in c.values()]

In [37]:
def delta(i,j):
    return 1 if i%(len(normalorder_C)//2)==j%(len(normalorder_C)//2) else 0

In [38]:
normalorder2_C=[(normalorder_C[j]*normalorder_C[i],0 if i==j else delta(i,j)-normalorder_C[i]*normalorder_C[j]) for i in range(len(normalorder_C)) for j in range(i,len(normalorder_C))]

In [39]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
    return expr1

# Op

In [24]:
# Kn=(K(0,1/sqrt(2),1/sqrt(2),g[1],gb[1]))
Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
# Kn=K(0,.1,sqrt(1-0.1**2),gb[1],g[2])
# Kn=K(.9,0,sqrt(1-0.9**2),gb[1],g[2])

In [35]:
expval=[(Dagger(ci)*ci,1) for ci in c.values()]

In [36]:
(Dagger(Kn)*Kn).expand().subs(selfjoint).subs(squareone).subs(normalorder2)

0.853553390593274 + 0.146446609406726*\gamma_2**2

In [37]:
(Kn).expand()

0.923879532511287 + 0.38268343236509*\bar{\gamma}_1*\gamma_2

In [218]:
(gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

-I*Dagger(c_1) + I*c_1

In [230]:
(I* g[1]* gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

1 - 2*Dagger(c_1)*c_1

In [231]:
(Kn).expand().subs(g2c).expand().subs(normalorder2_C).expand()

0.923879532511287 - 0.38268343236509*I + 0.76536686473018*I*Dagger(c_1)*c_1

## Op

In [50]:
# Kn=(K(0.9,sqrt(1-0.9**2),0,gb[1],g[2])) *(K(0.9,sqrt(1-0.9**2),0,gb[2],g[3]))
# Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
Kn=(K(0.1,sqrt(1-0.1**2)*sin(pi/4),sqrt(1-0.1**2)*cos(pi/4),gb[1],g[2]))


In [51]:
Kn

0.922721055122658 - 0.0191581945936129*I - I*(0.0462519732184832 - 0.382203575319017*I)*\bar{\gamma}_1*\gamma_2

In [52]:
recursiveapply((Dagger(Kn)*Kn).subs(g2c).expand(),normalorder2_C)

1.0 - 0.1*Dagger(c_1)*Dagger(c_2) - 0.1*Dagger(c_1)*c_2 - 0.1*Dagger(c_2)*c_1 + 0.1*c_1*c_2

In [53]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973514 - 0.703562363973515*I*Dagger(c_1)*Dagger(c_2) - 1.40712472794703*Dagger(c_1)*c_1 - 0.703562363973515*I*Dagger(c_1)*c_2 + 0.703562363973515*I*Dagger(c_2)*c_1 - 0.703562363973515*I*c_1*c_2

In [54]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973515 + 0.703562363973514*I*Dagger(c_1)*Dagger(c_2) - 1.40712472794703*Dagger(c_1)*c_1 + 0.703562363973514*I*Dagger(c_1)*c_2 - 0.703562363973514*I*Dagger(c_2)*c_1 + 0.703562363973514*I*c_1*c_2

In [55]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.1 + 1.0*Dagger(c_1)*Dagger(c_2) + 0.400000000000001*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 0.2*Dagger(c_1)*c_1 - 1.0*Dagger(c_1)*c_2 - 1.0*Dagger(c_2)*c_1 + 0.2*Dagger(c_2)*c_2 - 1.0*c_1*c_2

In [56]:
recursiveapply((Dagger(Kn)*(I*gb[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.1 + 1.0*Dagger(c_1)*Dagger(c_2) + 1.0*Dagger(c_1)*c_2 + 1.0*Dagger(c_2)*c_1 - 1.0*c_1*c_2

In [57]:
recursiveapply((Dagger(Kn)*(I*gb[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.703562363973515 - 0.703562363973514*I*Dagger(c_1)*Dagger(c_2) + 0.703562363973515*I*Dagger(c_1)*c_2 - 0.703562363973514*I*Dagger(c_2)*c_1 + 1.40712472794703*Dagger(c_2)*c_2 - 0.703562363973514*I*c_1*c_2

In [58]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973514 - 0.703562363973515*I*Dagger(c_1)*Dagger(c_2) + 0.703562363973515*I*Dagger(c_1)*c_2 - 0.703562363973515*I*Dagger(c_2)*c_1 - 1.40712472794703*Dagger(c_2)*c_2 - 0.703562363973515*I*c_1*c_2

In [308]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.784601809837321*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 - 1.11022302462516e-16*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 + 5.55111512312578e-17*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 + 0.784601809837321*I*Dagger(c_1)*Dagger(c_2)*c_1*c_3 + 2.77555756156289e-17*I*Dagger(c_1)*Dagger(c_2)*c_2*c_3 + 0.435889894354067*I*Dagger(c_1)*Dagger(c_3) - 0.784601809837321*I*Dagger(c_1)*Dagger(c_3)*c_1*c_2 - 0.784601809837321*I*Dagger(c_1)*c_1*c_2*c_3 + 0.435889894354067*I*Dagger(c_1)*c_3 + 0.392300904918661*I*Dagger(c_2)*Dagger(c_3) + 2.77555756156289e-17*I*Dagger(c_2)*Dagger(c_3)*c_1*c_2 + 5.55111512312578e-17*I*Dagger(c_2)*Dagger(c_3)*c_1*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*c_1 - 2.77555756156289e-17*I*Dagger(c_2)*c_1*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*c_2 + 0.392300904918661*I*Dagger(c_2)*c_3 - 0.435889894354067*I*Dagger(c_3)*c_1 - 5.5511151231257

In [309]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.81 + 0.9*Dagger(c_1)*Dagger(c_2) - 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 6.48*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 - 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 - 3.24*Dagger(c_1)*Dagger(c_2)*c_1*c_2 - 1.8*Dagger(c_1)*Dagger(c_2)*c_1*c_3 + 1.0*Dagger(c_1)*Dagger(c_3) - 1.8*Dagger(c_1)*Dagger(c_3)*c_1*c_2 - 3.24*Dagger(c_1)*Dagger(c_3)*c_1*c_3 - 1.8*Dagger(c_1)*Dagger(c_3)*c_2*c_3 - 1.62*Dagger(c_1)*c_1 + 1.8*Dagger(c_1)*c_1*c_2*c_3 - 0.9*Dagger(c_1)*c_2 - 1.0*Dagger(c_1)*c_3 + 0.9*Dagger(c_2)*Dagger(c_3) + 2.77555756156289e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 1.8*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 3.24*Dagger(c_2)*Dagger(c_3)*c_2*c_3 - 0.9*Dagger(c_2)*c_1 + 2.77555756156289e-17*Dagger(c_2)*c_1*c_2*c_3 - 1.62*Dagger(c_2)*c_2 - 0.9*Dagger(c_2)*c_3 - 1.0*Dagger(c_3)*c_1 + 1.8*Dagger(c_3)*c_1*c_2*c_3 - 0.9*Dagger(c_3)*c_2 - 1.62*Dagger(c_3)*c_3 - 0.9*c_1*c_2 - 1.0*c_1*c_3 - 0.9*c_2*c_3

In [310]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.392300904918661 + 1.38777878078145e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 2.22044604925031e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 - 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 1.11022302462516e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 - 5.55111512312578e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_2 - 2.77555756156289e-17*Dagger(c_1)*c_1*c_2*c_3 + 0.435889894354067*Dagger(c_2)*Dagger(c_3) + 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 1.56920361967464*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 2.77555756156289e-17*Dagger(c_2)*c_1*c_2*c_3 - 0.784601809837321*Dagger(c_2)*c_2 - 0.435889894354067*Dagger(c_2)*c_3 + 5.55111512312578e-17*Dagger(c_3)*c_1 - 0.435889894354067*Dagger(c_3)*c_2 - 0.784601809837321*Dagger(c_3)*c_3 + 5.55111512312578e-17*c_1*c_3 - 0.435889894354067*c_2*c_3

In [313]:
recursiveapply((Dagger(Kn)*(I*gb[2]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.9 + 0.81*Dagger(c_1)*Dagger(c_2) + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_3 - 1.8*Dagger(c_1)*Dagger(c_2)*c_2*c_3 + 0.9*Dagger(c_1)*Dagger(c_3) - 1.38777878078145e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_2 + 0.81*Dagger(c_1)*c_2 + 0.9*Dagger(c_1)*c_3 + 1.0*Dagger(c_2)*Dagger(c_3) - 1.8*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 1.66533453693773e-16*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 0.81*Dagger(c_2)*c_1 - 1.8*Dagger(c_2)*c_1*c_2*c_3 + 1.0*Dagger(c_2)*c_3 + 0.9*Dagger(c_3)*c_1 - 5.55111512312578e-17*Dagger(c_3)*c_1*c_2*c_3 + 1.0*Dagger(c_3)*c_2 + 2.77555756156289e-17*Dagger(c_3)*c_3 - 0.81*c_1*c_2 - 0.9*c_1*c_3 - 1.0*c_2*c_3

In [312]:
recursiveapply((Dagger(Kn)*(I*g[3]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.435889894354067 + 0.392300904918661*Dagger(c_1)*Dagger(c_2) + 5.55111512312578e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 + 1.11022302462516e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 0.784601809837321*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 5.55111512312578e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_2 + 1.38777878078145e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_3 + 0.784601809837321*Dagger(c_1)*Dagger(c_3)*c_2*c_3 - 2.77555756156289e-17*Dagger(c_1)*c_1*c_2*c_3 + 0.392300904918661*Dagger(c_1)*c_2 + 2.77555756156289e-17*Dagger(c_2)*Dagger(c_3) + 0.784601809837321*Dagger(c_2)*Dagger(c_3)*c_1*c_3 + 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 0.392300904918661*Dagger(c_2)*c_1 + 5.55111512312578e-17*Dagger(c_2)*c_3 + 0.784601809837321*Dagger(c_3)*c_1*c_2*c_3 + 5.55111512312578e-17*Dagger(c_3)*c_2 - 0.871779788708135*Dagger(c_3)*c_3 - 0.392300904918661*c_1*c_2

In [106]:
Kn.subs(g2c).expand().subs(squarezero).subs(normalorder2_C)

0.923879532511287 + 0.191341716182545*c_1*Dagger(c_1)

In [126]:
(Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

0.426776695296637*I*c_1*Dagger(c_1)

In [118]:
(Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C).subs(normalorder2_C)

0.213388347648318*I*Dagger(c_1)*Dagger(c_2) + 0.213388347648318*I*c_1*Dagger(c_2) + 0.213388347648318*I*c_1*c_2 - 0.213388347648318*I*c_2*Dagger(c_1)

In [119]:
(Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

0.213388347648318*I*Dagger(c_1)*Dagger(c_2) + 0.213388347648318*I*c_1*Dagger(c_2) - 0.213388347648318*I*c_1*c_2 + 0.0220970869120796*I*c_1**2*c_2*Dagger(c_1) + 0.213388347648318*I*c_2*Dagger(c_1)

In [44]:
N((1+I)**(0.5))

1.09868411346781 + 0.455089860562227*I

In [24]:
N(ea(0,1/sqrt(2),1/sqrt(2)))

0.923879532511287 - 0.38268343236509*I

In [17]:
N(exp(I*pi/8))

0.923879532511287 + 0.38268343236509*I

# Initial

In [73]:
# s=Dagger(c[1]-c[2])/sqrt(2)
s=(1-Dagger(c[1])*Dagger(c[2]))/sqrt(2)

In [74]:
recursiveapply((Dagger(s)*s).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*Dagger(c_2)/2 - Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 - Dagger(c_1)*c_1/2 - Dagger(c_2)*c_2/2 + c_1*c_2/2

In [75]:
recursiveapply((Dagger(s)*I*g[1]*gb[1]*s).expand().subs(g2c).expand(),normalorder2_C)

Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 - Dagger(c_1)*c_1/2 + Dagger(c_2)*c_2/2 - c_1*c_2/2

In [76]:
recursiveapply((Dagger(s)*I*g[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

I*Dagger(c_1)*Dagger(c_2)/2 + I*Dagger(c_1)*c_2/2 - I*Dagger(c_2)*c_1/2 + I*c_1*c_2/2

In [77]:
recursiveapply((Dagger(s)*I*g[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-1 + Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2 + Dagger(c_1)*c_1 - Dagger(c_1)*c_2/2 - Dagger(c_2)*c_1/2 + Dagger(c_2)*c_2 - c_1*c_2/2

In [78]:
recursiveapply((Dagger(s)*I*gb[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-1 + Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2 + Dagger(c_1)*c_1 + Dagger(c_1)*c_2/2 + Dagger(c_2)*c_1/2 + Dagger(c_2)*c_2 - c_1*c_2/2

In [79]:
recursiveapply((Dagger(s)*I*gb[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-I*Dagger(c_1)*Dagger(c_2)/2 + I*Dagger(c_1)*c_2/2 - I*Dagger(c_2)*c_1/2 - I*c_1*c_2/2

In [80]:
recursiveapply((Dagger(s)*I*g[2]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 + Dagger(c_1)*c_1/2 - Dagger(c_2)*c_2/2 - c_1*c_2/2

In [56]:
s0=Dagger(c[1]-c[2])/sqrt(2)

In [57]:
recursiveapply((Dagger(s0)*s0).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*c_1/2 + Dagger(c_1)*c_2/2 + Dagger(c_2)*c_1/2 - Dagger(c_2)*c_2/2

In [58]:
recursiveapply((Dagger(s0)*I*g[1]*gb[1]*s0).expand().subs(g2c).expand(),normalorder2_C)

-Dagger(c_1)*Dagger(c_2)*c_1*c_2 - Dagger(c_1)*c_1/2 - Dagger(c_1)*c_2/2 - Dagger(c_2)*c_1/2 - Dagger(c_2)*c_2/2

In [ ]:
recursiveapply((Dagger(s0)*I*g[1]*g[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

0

In [56]:
recursiveapply((Dagger(s0)*I*g[1]*gb[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*Dagger(c_2)*c_1*c_2 - Dagger(c_1)*c_1 - Dagger(c_2)*c_2

In [86]:
recursiveapply((g[1]*g[2]*(I*gb[1]*I*gb[2])).expand().subs(g2c).expand(),normalorder2_C)

-1 + 4*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 2*Dagger(c_1)*c_1 + 2*Dagger(c_2)*c_2